In [1]:
from teadata import DataEngine
import pandas as pd

repo = DataEngine.from_snapshot(search=True)  # auto-discovers the packaged snapshot


In [2]:
campus_df = repo.campuses.to_df()
district_df = repo.districts.to_df()[["id", "name", "district_number"]]

campus_df = (
    campus_df
    .merge(
        district_df.rename(columns={
            "id": "district_id_merge",
            "name": "district_name"
        }),
        left_on="district_id",
        right_on="district_id_merge",
        how="left",
    )
    .drop(columns=["district_id_merge"])
)


In [3]:
# Friendly renames for reporting
campus_df = campus_df.rename(columns={
    "name": "campus_name",
    "campus_number": "campus_number",
    "overall_rating_2025": "overall_rating_2025",
    "grade_range": "grade_levels_served",
    "school_type": "school_type",
    "campus_2024_student_enrollment_econ_disadv_percent": "pct_econ_disadv",
    "campus_2024_student_enrollment_special_ed_percent": "pct_special_ed",
    "campus_2024_student_membership_2022_23_attrition_all_students_percent": "pct_attrition",
    "campus_2024_student_membership_2023_mobility_all_students_percent": "pct_mobility",
    "campus_2024_student_enrollment_at_risk_percent": "pct_at_risk",
    "campus_2024_student_enrollment_el_percent": "pct_emergent_bilingual",
    "campus_2024_staff_teacher_beginning_full_time_equiv_percent": "pct_beginning_teachers",
})

# Derive district/charter label
campus_df["governance_type"] = campus_df["is_charter"].map({True: "Charter", False: "District"})

# Optional: derive county from the six-digit district number (first three digits are the county code)
campus_df["county_code"] = campus_df["district_number"].str.replace("'", "", regex=False).str[:3]
# campus_df["county_name"] = campus_df["county_code"].map(COUNTY_CODE_TO_NAME)  # supply your TEA county lookup dict

numeric_cols = [
    "pct_econ_disadv", "pct_special_ed", "pct_attrition", "pct_mobility",
    "pct_at_risk", "pct_emergent_bilingual", "pct_beginning_teachers"
]
campus_df[numeric_cols] = campus_df[numeric_cols].apply(pd.to_numeric, errors="coerce")


KeyError: 'district_number'

In [ ]:
base_columns = [
    "campus_number", "campus_name", "district_name", "county_code",
    "governance_type", "grade_levels_served", "school_type",
    "overall_rating_2025", "pct_econ_disadv", "pct_special_ed", "pct_attrition",
    "pct_mobility", "pct_at_risk", "pct_emergent_bilingual",
    "pct_beginning_teachers"
]

district_campuses = campus_df.loc[campus_df["governance_type"] == "District", base_columns].copy()
charter_campuses = campus_df.loc[campus_df["governance_type"] == "Charter", base_columns].copy()


In [ ]:
district_campuses.to_excel("campus_stats_district.xlsx", index=False)
charter_campuses.to_excel("campus_stats_charter.xlsx", index=False)

# or preview in notebook
district_campuses.head()
charter_campuses.head()
